In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# Memuat dataset
dataset = pd.read_csv('../data/raw/weather_classification_data.csv')



In [18]:
# 1. Memeriksa nilai hilang
# Cek jumlah nilai hilang di setiap kolom
print(dataset.isnull().sum())

# Menangani nilai hilang: Menggunakan SimpleImputer untuk mengisi nilai hilang
# Misalnya, menggunakan strategi 'mean' untuk kolom numerik
imputer = SimpleImputer(strategy='mean')
dataset_imputed = pd.DataFrame(imputer.fit_transform(dataset.select_dtypes(include=['float64', 'int64'])))

# Menangani kolom kategorikal dengan mengganti nilai hilang menggunakan modus
categorical_columns = dataset.select_dtypes(include=['object']).columns
for col in categorical_columns:
    dataset[col].fillna(dataset[col].mode()[0], inplace=True)

# 2. Mengonversi variabel kategorikal
# Jika menggunakan Label Encoding untuk kolom kategorikal
label_encoder = LabelEncoder()
for col in categorical_columns:
    dataset[col] = label_encoder.fit_transform(dataset[col])


Temperature             0
Humidity                0
Wind Speed              0
Precipitation (%)       0
Cloud Cover             0
Atmospheric Pressure    0
UV Index                0
Season                  0
Visibility (km)         0
Location                0
Weather Type            0
dtype: int64


In [25]:
X = dataset.drop(columns=['Weather Type'])
y = dataset['Weather Type']

# 4. Membagi data menjadi data latih dan data uji (80% data latih dan 20% data uji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [26]:
import joblib

# Menyimpan data yang sudah diproses ke file
joblib.dump(X_train, 'X_train.pkl')
joblib.dump(X_test, 'X_test.pkl')
joblib.dump(y_train, 'y_train.pkl')
joblib.dump(y_test, 'y_test.pkl')


['y_test.pkl']

In [28]:
import joblib

# Menyimpan nama kolom fitur
joblib.dump(X.columns, 'feature_names.pkl')
# Menyimpan y_train
joblib.dump(y_train, 'y_train.pkl')


['y_train.pkl']

In [33]:
import pandas as pd

# Memastikan bahwa y_train adalah pandas Series dengan indeks yang sama dengan dataset
y_train_series = pd.Series(y_train, index=dataset.index)

# Memfilter dataset untuk hanya menampilkan data dengan Class 2
class_2_data = dataset[y_train_series == 2]  # Mengambil data yang berkelas 2

# Menampilkan data yang sesuai dengan Class 2
print(class_2_data)
# Menampilkan distribusi kelas cuaca
print(dataset['Weather Type'].value_counts())


       Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
6             -2.0        97         8.0               86.0            2   
7              3.0        85         6.0               96.0            3   
13           -10.0        87        15.0               67.0            2   
20           -10.0        67         8.5               75.0            2   
21            -7.0        88        12.5               98.0            2   
...            ...       ...         ...                ...          ...   
13177         -3.0        94         8.0               50.0            2   
13179         -4.0        98         1.5               76.0            2   
13181         -7.0        97         0.5               99.0            2   
13187          4.0        71        18.5               88.0            2   
13196         -1.0        76         3.5               23.0            1   

       Atmospheric Pressure  UV Index  Season  Visibility (km)  Location  \
6          

In [34]:
print(dataset.columns)

# Menampilkan distribusi kelas cuaca (Weather Type)
print(dataset['Weather Type'].value_counts())


Index(['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)',
       'Cloud Cover', 'Atmospheric Pressure', 'UV Index', 'Season',
       'Visibility (km)', 'Location', 'Weather Type'],
      dtype='object')
Weather Type
1    3300
0    3300
3    3300
2    3300
Name: count, dtype: int64


In [36]:
# Menyaring data berdasarkan nilai Weather Type
class_0_data = dataset[dataset['Weather Type'] == 0]  # Data dengan Weather Type = 0
class_1_data = dataset[dataset['Weather Type'] == 1]  # Data dengan Weather Type = 1
class_2_data = dataset[dataset['Weather Type'] == 2]  # Data dengan Weather Type = 2
class_3_data = dataset[dataset['Weather Type'] == 3]  # Data dengan Weather Type = 3

# Menampilkan beberapa data untuk melihat apa yang diprediksi oleh masing-masing kelas
print("Data dengan Weather Type = 0")
print(class_0_data.head())

print("\nData dengan Weather Type = 1")
print(class_1_data.head())

print("\nData dengan Weather Type = 2")
print(class_2_data.head())

print("\nData dengan Weather Type = 3")
print(class_3_data.head())


Data dengan Weather Type = 0
    Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
1          39.0        96         8.5               71.0            3   
5          32.0        55         3.5               26.0            2   
10         35.0        45         6.0               86.0            3   
12         12.0        59        10.5               25.0            3   
15         10.0        50         6.5               46.0            3   

    Atmospheric Pressure  UV Index  Season  Visibility (km)  Location  \
1                1011.43         7       1             10.0         1   
5                1010.03         2       2              5.0         1   
10                879.88         2       1              1.0         2   
12               1016.08         3       0              5.5         2   
15               1000.44         2       2              8.5         2   

    Weather Type  
1              0  
5              0  
10             0  
12             0 

In [53]:
import pandas as pd

# Ambil 300 data acak dari dataset
sampled_data = dataset.sample(n=300, random_state=42)  # random_state untuk memastikan hasil yang konsisten

# Menampilkan 300 data acak yang diambil
print(sampled_data)



       Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
4111          23.0        38         4.5                6.0            0   
10607         62.0        94        14.5               83.0            0   
7372          40.0        51         2.0               12.0            3   
11786         -7.0        79         6.5               90.0            2   
12227         21.0        94        10.0              109.0            3   
...            ...       ...         ...                ...          ...   
11292         30.0        60         8.5               20.0            2   
9063          34.0        76        10.5               34.0            3   
4695          -5.0        86        10.5               92.0            2   
6906           8.0        40        10.0               73.0            2   
9427         -13.0       100        22.0               81.0            3   

       Atmospheric Pressure  UV Index  Season  Visibility (km)  Location  \
4111       

In [54]:
# Misalnya, kolom 'Weather Type' adalah target (y), dan sisanya adalah fitur (X)
X = sampled_data.drop(columns=['Weather Type'])  # Fitur
y = sampled_data['Weather Type']  # Target



In [47]:
# Menyaring data untuk melihat tipe cuaca untuk masing-masing kelas
class_0_data = sampled_data[sampled_data['Weather Type'] == 0]  # Class 0
class_1_data = sampled_data[sampled_data['Weather Type'] == 1]  # Class 1
class_2_data = sampled_data[sampled_data['Weather Type'] == 2]  # Class 2
class_3_data = sampled_data[sampled_data['Weather Type'] == 3]  # Class 3

# Menampilkan beberapa contoh data untuk setiap kelas
print("Data untuk Class 0 (Weather Type = 0):")
print(class_0_data.head())

print("Data untuk Class 1 (Weather Type = 1):")
print(class_1_data.head())

print("Data untuk Class 2 (Weather Type = 2):")
print(class_2_data.head())

print("Data untuk Class 3 (Weather Type = 3):")
print(class_3_data.head())


Data untuk Class 0 (Weather Type = 0):
       Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
5156          25.0        58        13.0               36.0            3   
4618          11.0        29        13.0               77.0            3   
10642         21.0        69         0.0               40.0            3   
6205          17.0        76         4.0               39.0            3   
7503          30.0        66        11.5               20.0            2   

       Atmospheric Pressure  UV Index  Season  Visibility (km)  Location  \
5156                1011.99         4       0              5.5         1   
4618                 876.06        12       0              7.0         2   
10642               1001.43         2       2              7.0         1   
6205                1010.02         4       0              6.5         0   
7503                1013.21         1       2              5.0         1   

       Weather Type  
5156              0  
461

In [55]:
sampled_data.to_csv('sampled_data.csv', index=False)
